# Web - scraping danych z platformy OLX

## Importowanie niezbędnich bibliotek

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

## Pobranie danych

### Pobranie ofert dla miast Katowice na datę 01.12.2023

In [2]:
# I. Pobranie ofert dla miasta Katowice na datę 01.12.2023

# Na tym eatpie zostaną pobrane podstawowe infomacje na temat ofert takie jak:

#       Nazwa oferty
#       Lokalizacja
#       Data dodania
#       Powierzchnia
#       Cena za m2
#       Kwota (łączna)
#       Zdjęcie
#       Link

def scrape_olx_page(url):
    resp = requests.get(url)
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'html.parser')
        offers = []

        for item in soup.select("div[data-cy='l-card']"):
            title_node = item.select_one("h6.css-16v5mdi.er34gjf0")
            location_date_node = item.select_one("p.css-veheph.er34gjf0")
            area_price_node = item.select_one("span.css-643j0o")
            total_price_node = item.select_one("p.css-10b0gli.er34gjf0")
            link_node = item.select_one("a.css-rc5s2u")

            title = title_node.text.strip() if title_node else None
            total_price = total_price_node.text.strip() if total_price_node else None
            link = urljoin(url, link_node['href']) if link_node else None

            location_date_text = location_date_node.text.strip() if location_date_node else None
            if location_date_text:
                location, date_added = map(str.strip, location_date_text.rsplit('-', 1))
            else:
                location, date_added = None, None

            area_price_text = area_price_node.text.strip() if area_price_node else None
            if area_price_text:
                area, price_per_m2 = map(str.strip, area_price_text.split('-', 1))
            else:
                area, price_per_m2 = None, None

            offer = {
                "Nazwa oferty": title,
                "Lokalizacja": location,
                "Data dodania": date_added,
                "Powierzchnia w m²": area,
                "Cena za 1m²": price_per_m2,
                "Kwota": total_price,
                "Link": link
            }

            offers.append(offer)

        return offers

    else:
        print(f"Failed to fetch the page. Status code: {resp.status_code}")
        return []

# Nastepnie do głebszej analizy niezbedne jest pobranie również atrybutów bezpośrednio zawartych w danych ofercie. 

# Ze względu na to iż do Grupy OLX należy również serwis Otodom, na platformie OLX wyswietlają się oferty, które po kliknięci przenoszą nas bezpośrednio do strony Otodom, której struktura rózni się od stuktury strony OLX
# W związku z tym, próba pobrania ofert, które znajdują się na stronie Oto - nie powiedziedzie się.Konieczne jest zdefiniowanie dwóch funkcji: 
#       Jedna - która będzie pobierała szczegóły oferty ze strony OLX
#       Druga - która będzie pobierała szczegóły oferty ze strony OTODom 

# Na tym etapie załozóno również, ze aby móc porównywac oferty, których struktury się różnią, nalezy wybrac atrybuty unikalne dla oby rodzajów ofert.
# W wyniku tej weryfikacji ustalono , że do analizy przechodzą cechy takie jak:
#        Poziom (Piętro), 
#        Umeblowane (Czy mieskzanie jest umeblowane), 
#        Liczba pokoi oraz Opis oferty.
# Dane te są popbierne już bezpośrednio z konkretnej oferty.



def scrape_offer_details_olx(offer_url):
    resp = requests.get(offer_url)
    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'html.parser')
        attributes = soup.select("ul.css-sfcl1s li p.css-b5m1rv.er34gjf0")
        offer_attributes = {}

        # Dodajemy kod do wydobycia opisu oferty
        description_node = soup.select_one("div.css-1t507yq.er34gjf0")
        description = description_node.text.strip() if description_node else None
        offer_attributes["Opis"] = description

        for attr in attributes:
            parts = attr.text.split(":")
            if len(parts) >= 2:
                key = parts[0].strip()
                value = parts[1].strip()
                if key in ["Poziom", "Umeblowane", "Liczba pokoi"]:
                    offer_attributes[key] = value

        return offer_attributes

    else:
        print(f"Failed to fetch offer page. Status code: {resp.status_code}")
        return None

def scrape_offer_details_otodom(offer_url):
    resp = requests.get(offer_url, headers={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    })

    if resp.status_code == 200:
        soup = BeautifulSoup(resp.text, 'html.parser')

        construction_status_node = soup.select_one("div[data-testid='table-value-construction_status']")
        rooms_num_node = soup.select_one("div[data-testid='table-value-rooms_num']")
        floor_node = soup.select_one("div[data-testid='table-value-floor']")

        construction_status = construction_status_node.text.strip() if construction_status_node else None
        rooms_num = rooms_num_node.text.strip() if rooms_num_node else None
        floor = floor_node.text.strip() if floor_node else None

        description_node = soup.select_one("div.css-1wekrze.e1lbnp621")
        description = description_node.text.strip() if description_node else None

        offer_attributes = {
            "Poziom": floor,
            "Umeblowane": construction_status,
            "Liczba pokoi": rooms_num,
            "Opis": description
        }

        return offer_attributes

    else:
        print(f"Failed to fetch the page. Status code: {resp.status_code}")
        return None


url_to_scrape_base = "https://www.olx.pl/nieruchomosci/mieszkania/sprzedaz/katowice/q-mieszkanie/?page={}"

# Celem analizy jest zebranie wszystkich ofert dla miasta Katowice, natomiast na jednej stronie znajduje się zaledwie kilkadziesiąt ofert.
# W związku z tym nalezy utworzyć olgorytm który będzie iterował, przez wszystkie dostępne strony na datę 01.12.2023  - 15 stron 

# 1. Iterowanie przez wszystkie strony i zebranie ofert

total_offers = []

for page_num in range(1, 16): 
    url_to_scrape = url_to_scrape_base.format(page_num)
    total_offers += scrape_olx_page(url_to_scrape)

print(f"\nLiczba wszystkich ofert: {len(total_offers)}") # Sprawdzamy ile jest wszystkich ofert

df_offers = pd.DataFrame(total_offers)

# 2. Zebranie szczegółowych atrybutów dla ofert OLX oraz OTOdom

all_attributes = []

for index, row in df_offers.iterrows():
    offer_url = row['Link']

    if offer_url:
        if "otodom.pl" in offer_url:
            offer_attributes = scrape_offer_details_otodom(offer_url)
            if offer_attributes:
                all_attributes.append(offer_attributes)
            else:
                print(f"\nBrak atrybutów dla oferty {index + 1} ({offer_url})")
        else:
            offer_attributes = scrape_offer_details_olx(offer_url)
            if offer_attributes:
                all_attributes.append(offer_attributes)
            else:
                print(f"\nBrak atrybutów dla oferty {index + 1} ({offer_url})")
    else:
        print(f"\nBrak adresu URL dla oferty {index + 1}")

# Tworzenie DataFrame z atrybutów
df_attributes = pd.DataFrame(all_attributes)




Liczba wszystkich ofert: 813
Failed to fetch the page. Status code: 410

Brak atrybutów dla oferty 447 (https://www.otodom.pl/pl/oferta/premium-bezposrednio-cicha-okolica-brynow-ID4nzLT.html)
Failed to fetch the page. Status code: 410

Brak atrybutów dla oferty 692 (https://www.otodom.pl/pl/oferta/4-pokojowe-mieszkanie-98m2-loggia-bez-prowizji-ID4g352.html)


### Zapisanie danych do plików CSV

In [3]:
# Połącznie danych na temat ofert oraz szczegółowych atrybutów
merged_df = pd.concat([df_offers, df_attributes], axis=1)

In [4]:
liczba_ofert_z_liczba_pokoi_1 = len(merged_df[merged_df['Liczba pokoi'] == '1'])
liczba_ofert_z_liczba_pokoi_1

48

In [5]:
merged_df.to_csv('merged_df_13g.csv')